In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

np.random.seed(20201229)
tf.set_random_seed(20201229)

In [2]:
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
class SingleCNN:
    def __init__(self, num_filters, num_units):
        with tf.Graph().as_default():
            self.prepare_model(num_filters, num_units)
            self.prepare_session()

    def prepare_model(self, num_filters, num_units):
        num_units1 = 14*14*num_filters
        num_units2 = num_units
        
        with tf.name_scope('input'):
            x = tf.placeholder(tf.float32, [None, 784], name='input')
            x_image = tf.reshape(x, [-1,28,28,1])

        with tf.name_scope('convolution'):
            W_conv = tf.Variable(tf.truncated_normal([5,5,1,num_filters], stddev=0.1), name='conv-filter')
            h_conv = tf.nn.conv2d(x_image, W_conv, strides=[1,1,1,1], padding='SAME', name='filter-output')

        with tf.name_scope('pooling'):            
            h_pool =tf.nn.max_pool(h_conv, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME', name='max-pool')
            h_pool_flat = tf.reshape(h_pool, [-1, 14*14*num_filters], name='pool-output')

        with tf.name_scope('fully-connected'):
            w2 = tf.Variable(tf.truncated_normal([num_units1, num_units2]))
            b2 = tf.Variable(tf.zeros([num_units2]))
            hidden2 = tf.nn.relu(tf.matmul(h_pool_flat, w2) + b2, name='fc-output')

        with tf.name_scope('softmax'):
            w0 = tf.Variable(tf.zeros([num_units2, 10]))
            b0 = tf.Variable(tf.zeros([10]))
            p = tf.nn.softmax(tf.matmul(hidden2, w0) + b0, name='softmax-output')
            
        with tf.name_scope('optimizer'):
            t = tf.placeholder(tf.float32, [None, 10], name='labels')
            loss = -tf.reduce_sum(t * tf.log(p), name='loss')
            train_step = tf.train.AdamOptimizer(0.0005).minimize(loss)
            
        with tf.name_scope('evaluator'):
            correct_prediction = tf.equal(tf.argmax(p, 1), tf.argmax(t, 1))
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')
            
        tf.summary.scalar("loss", loss)
        tf.summary.scalar("accuracy", accuracy)
        tf.summary.histogram("convolution_filters", W_conv)
        
        self.x, self.t, self.p = x, t, p
        self.train_step = train_step
        self.loss = loss
        self.accuracy = accuracy
        
    def prepare_session(self):
        sess = tf.Session()
        sess.run(tf.initialize_all_variables())
        summary = tf.summary.merge_all()
        writer = tf.summary.FileWriter("/tmp/mnist_df_logs", sess.graph)
        
        self.sess = sess
        self.summary = summary
        self.writer = writer

In [4]:
cnn = SingleCNN(16, 1024)

i = 0
for _ in range(4000):
    i += 1
    batch_xs, batch_ts = mnist.train.next_batch(100)
    cnn.sess.run(cnn.train_step, feed_dict={cnn.x:batch_xs, cnn.t:batch_ts})
    if i % 50 == 0:
        summary, loss_val, acc_val = cnn.sess.run(
            [cnn.summary, cnn.loss, cnn.accuracy],
            feed_dict={cnn.x:mnist.test.images, cnn.t:mnist.test.labels})
        print ('Step: %d, Loss: %f, Accuracy: %f'
               % (i, loss_val, acc_val))
        cnn.writer.add_summary(summary, i)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Step: 50, Loss: 2917.952637, Accuracy: 0.914500
Step: 100, Loss: 2646.487793, Accuracy: 0.919500
Step: 150, Loss: 1962.495361, Accuracy: 0.942900
Step: 200, Loss: 1649.287231, Accuracy: 0.951000
Step: 250, Loss: 1435.266113, Accuracy: 0.958700
Step: 300, Loss: 1227.430420, Accuracy: 0.964400
Step: 350, Loss: 1161.642334, Accuracy: 0.964600
Step: 400, Loss: 1087.458618, Accuracy: 0.967700
Step: 450, Loss: 1120.647705, Accuracy: 0.965900
Step: 500, Loss: 1039.302246, Accuracy: 0.968000
Step: 550, Loss: 995.620239, Accuracy: 0.969400
Step: 600, Loss: 946.867188, Accuracy: 0.970500
Step: 650, Loss: 988.792419, Accuracy: 0.969700
Step: 700, Loss: 912.168274, Accuracy: 0.972800
Step: 750, Loss: 920.776978, Accuracy: 0.971800
Step: 800, Loss: 794.952087, Accuracy: 0.976500
Step: 850, Loss: 963.596985, Accuracy: 0.970400
Step: 900, Loss: 862.485168, Accuracy: 0.971900
Step: 950, Loss: 771.258667, Accuracy: 0.976500
Step: